In [1]:
from PyQt5 import QtWidgets,uic
import sqlite3 as sql

In [2]:
def login():
    
    phone = homePage.usernameEnter.text()
    password = homePage.passwordEnter.text()
    
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_name,cust_Account_num,cust_email,cust_balance
            FROM customer_details WHERE cust_phone = ? AND cust_password = ?"""
    
    ret = con.execute(query,(phone,password))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        homePage.close()
        custDashboard.welcome.setText('Welcome,'+ret[0][0])
        custDashboard.info.setText('''
        Account num : {}
        Balance : {}'''.format(ret[0][1],ret[0][3]))
        custDashboard.show()
    else:
        homePage.message.setText('invalid phone or password')
        homePage.usernameEnter.setText('')
        homePage.passwordEnter.setText('')
    

def gotosignup():
    homePage.close()
    signupPage.show()
    
def backfromsignup():
    signupPage.close()
    homePage.show()
    
def register():
    
    cust_name = signupPage.name.text()
    cust_password = signupPage.password.text()
    cust_email = signupPage.email.text()
    cust_phone = signupPage.phone.text()
    cust_dob = signupPage.dob.text()
    
    cust_account_num = cust_dob.split('-')[0]+cust_dob.split('-')[1]+cust_dob.split('-')[2]+cust_phone[-5:]
    
    cust_Id = cust_account_num[:5]+cust_name[:5]
    
    con = sql.connect('demobank.db')
    
    query = """INSERT INTO customer_details(cust_Id,cust_name,
    cust_Account_num, cust_email, cust_phone, cust_password,
    cust_dob,cust_balance) VALUES(?,?,?,?,?,?,?,?)"""
    
    con.execute(query,(cust_Id,cust_name,cust_account_num,cust_email,
                      cust_phone,cust_password,cust_dob,"2000.0"))
    con.commit()
    con.close()
    
    signupPage.name.setText(" ")
    signupPage.password.setText(" ")
    signupPage.email.setText(" ")
    signupPage.phone.setText(" ")
    signupPage.dob.setText(" ")
    signupPage.message.setText('registration successful')

def custlogout():
    custDashboard.close()
    homePage.usernameEnter.setText('')
    homePage.passwordEnter.setText('')
    homePage.show()

def opentransfer():
    transfer_diag.show()
    
def transfer():
    
    phone = transfer_diag.lineEdit.text()
    password = transfer_diag.lineEdit_2.text()
    amount = transfer_diag.lineEdit_4.text()
    payee_ac_num = transfer_diag.lineEdit_3.text()
    
    
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_name,cust_Account_num,cust_email,cust_balance
            FROM customer_details WHERE cust_phone = ? AND cust_password = ?"""
    
    ret = con.execute(query,(phone,password))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        cust_bal = float(ret[0][3])
        
        if cust_bal >= float(amount):
            
            query = """SELECT cust_balance FROM customer_details WHERE cust_Account_num=?"""
            ret = con.execute(query,(payee_ac_num,))
            ret = ret.fetchall()
            if len(ret)>0:
                payee_bal = float(ret[0][0])

            cust_new_bal = cust_bal-float(amount)
            payee_new_bal = payee_bal + float(amount)
            
            query = """UPDATE customer_details SET cust_balance = ? WHERE cust_phone = ?"""
            con.execute(query,(str(cust_new_bal),phone))
            
            query = """UPDATE customer_details SET cust_balance = ? WHERE cust_Account_num = ?"""
            con.execute(query,(str(payee_new_bal),payee_ac_num))
            con.commit()
            con.close()
        else:
            con.close()
    else:
        con.close()
        
    
    

In [3]:
app = QtWidgets.QApplication([])

homePage = uic.loadUi('home.ui')
signupPage = uic.loadUi('signup.ui')
custDashboard = uic.loadUi('custDashboard.ui')
transfer_diag = uic.loadUi('transfer_dialog.ui')

homePage.show()

homePage.loginBut.clicked.connect(login)
homePage.signupBut.clicked.connect(gotosignup)

signupPage.back.clicked.connect(backfromsignup)
signupPage.signup.clicked.connect(register)


custDashboard.logout.clicked.connect(custlogout)
custDashboard.pushButton.clicked.connect(opentransfer)

transfer_diag.pushButton.clicked.connect(transfer)
app.exec()

0